<a href="https://colab.research.google.com/github/paddy3696/CNN_Pytorch/blob/main/inat_cnn_pretrained.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sat Apr 17 20:33:56 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchvision import datasets, models, transforms
from torch.utils.data.sampler import SubsetRandomSampler
import os

import matplotlib.pyplot as plt

In [3]:
%pip install wandb -q

# Ignore excessive warnings
import logging
logging.propagate = False 
logging.getLogger().setLevel(logging.ERROR)

# WandB – Import the wandb library
import wandb
#wandb.init(project='mnist_classification', entity='paddy3696')

     |████████████████████████████████| 2.1MB 9.4MB/s 
     |████████████████████████████████| 102kB 10.7MB/s 
     |████████████████████████████████| 133kB 46.4MB/s 
     |████████████████████████████████| 163kB 50.5MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 


In [4]:
train_gpu = torch.cuda.is_available()

if not train_gpu:
    print('CUDA is not available. Training on CPU')
else:
    print('CUDA is available! Training on GPU')

CUDA is available! Training on GPU


In [5]:
!pip install wget

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=ade11c972cf8349e0ffd153bb83421acbc1bc4f0008004496690b753065473fc
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget
wget.download('https://storage.googleapis.com/wandb_datasets/nature_12K.zip')
!unzip /content/nature_12K.zip

In [7]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    model = None
    input_size = 0

    if model_name == "resnet":
        model = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        model = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        model = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.classifier[6].in_features
        model.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        model = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        model.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        model = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        num_ftrs = model.classifier.in_features
        model.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        model = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model, feature_extract)
        # Handle the auxilary net
        num_ftrs = model.AuxLogits.fc.in_features
        model.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model, input_size

In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [9]:
def fit(model,train_loader,val_loader,optimizer,epoch,criterion, is_inception=False):
  acc_epoch = []

  for epoch in range(1, epoch+1):
    train_loss = 0.0
    val_loss = 0.0
    class_correct = list(0. for i in range(10))
    class_total = list(0. for i in range(10))
    print('training started')

    model.train()
    for data, target in train_loader:
      if train_gpu:
        data, target = data.cuda(), target.cuda()
      
      optimizer.zero_grad()
      if is_inception:
        output, aux_output = model(data)
        loss1 = criterion(output, target)
        loss2 = criterion(aux_output, target)
        loss = loss1 + 0.4*loss2
      else:
        output = model(data)
        loss = criterion(output, target)
      
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*data.size(0)

    train_loss = train_loss/len(train_loader.sampler)
    print('Training over')
    model.eval()
    for data, target in val_loader:
      batch_data_size = data.size(0)
      if train_gpu:
        data, target = data.cuda(), target.cuda()
      output = model(data)
      loss = criterion(output, target)
      val_loss += loss.item()*data.size(0)

      _, pred = torch.max(output, 1)    
      correct_tensor = pred.eq(target.data.view_as(pred))
      correct = np.squeeze(correct_tensor.numpy()) if not train_gpu else np.squeeze(correct_tensor.cpu().numpy())
      for i in range(batch_data_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1
    
    val_loss = val_loss/len(val_loader.sampler)
    val_accuracy = 100. * np.sum(class_correct) / np.sum(class_total)
    acc_epoch.append(val_accuracy)
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, val_loss))
    print('Val Accuracy (Overall): %.2f%% (%2d/%2d)\n' % (val_accuracy,np.sum(class_correct), np.sum(class_total)))

    wandb.log({"Train Loss": train_loss,"Val Loss": val_loss,"Val Accuracy": val_accuracy})
  
  acc_best = max(acc_epoch)
  wandb.log({'accuracy' : acc_best})
  #return loss_train, loss_val

In [10]:
sweep_config = {
    'method': 'bayes', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'model_name': {
            'values': ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']
        },
        'batch_size': {
            'values': [16, 32]
        },
        'epoch': {
            'values': [10,20]
        },
        'data_aug': {
            'values': ['Yes', 'No']
        },
        'optimizer': {
            'values': ['SGD','ADAM'] 
        },
        'lr': {
            'values': [0.1,0.01, 0.001] 
        },
    }
}

In [11]:
#sweep_id = wandb.sweep(sweep_config, entity="paddy3696", project="cnn_inat")

In [12]:
def train():
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'model_name': 'inception',
      'epoch': 10,
      'data_aug': 'Yes',
      'optimizer': 'SGD',
      'batch_size': 16,
      'seed': 9,
      'lr': 0.001,
  }

  # Initialize a new wandb run
  wandb.init(config=config_defaults)
   
  # Config is a variable that holds and saves hyperparameters and inputs
  config = wandb.config
  
  #config.model_name = 'inception'
  model_name = config.model_name
  num_classes = 10
  feature_extract = False 

  model, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

  print(model)
  
  data_dir = '/content/inaturalist_12K'
                              
  if config.data_aug == 'Yes':
    train_transforms = transforms.Compose([transforms.RandomRotation(30),transforms.RandomResizedCrop(input_size),
                                       transforms.RandomHorizontalFlip(),transforms.ToTensor(), 
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
                                           
  elif config.data_aug == 'No':
    train_transforms = transforms.Compose([transforms.RandomResizedCrop(input_size), transforms.ToTensor()])

  train_data = datasets.ImageFolder(data_dir + '/train',transform=train_transforms)

  batch_size = config.batch_size
  validation_split = .1
  shuffle_dataset = True
  random_seed= config.seed

  dataset_size = len(train_data)
  indices = list(range(dataset_size))
  split = int(np.floor(validation_split * dataset_size))
  if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
  train_indices, val_indices = indices[split:], indices[:split]

  train_sampler = SubsetRandomSampler(train_indices)
  valid_sampler = SubsetRandomSampler(val_indices)

  train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=train_sampler)
  val_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,sampler=valid_sampler)
  print('data ready')
    

  if train_gpu:
    model.cuda()

  criterion = nn.CrossEntropyLoss()
  epoch = config.epoch

  params_to_update = model.parameters()
  print("Params to learn:")
  if feature_extract:
    params_to_update = []
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
  else:
    for name,param in model.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

  if config.optimizer == 'SGD':
    optimizer = optim.SGD(params_to_update, lr=config.lr,momentum=0.9)
  elif config.optimizer == 'ADAM':
    optimizer = optim.Adam(model.parameters(), lr=config.lr)

  wandb.run.name = str(config.model_name) + '_bs_' + str(config.batch_size)

  fit(model,train_loader,val_loader,optimizer,epoch,criterion,is_inception=(model_name=="inception"))

In [13]:
#wandb.agent(sweep_id, train,count=25)
wandb.agent("5utort2o", entity="paddy3696",project="cnn_inat", function =train,count=100)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Agent Starting Run: iceury0y with config:
wandb: 	batch_size: 32
wandb: 	data_aug: No
wandb: 	epoch: 20
wandb: 	lr: 0.001
wandb: 	model_name: densenet
wandb: 	optimizer: SGD
wandb: Currently logged in as: paddy3696 (use `wandb login --relogin` to force relogin)


Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth



DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (rel

wandb: Ctrl + C detected. Stopping sweep.
